# **CRUSHgebra**

The goal of this project is to design and train a single neural network that can perform two different tasks simultaneously (Multi-Task Learning).
1. Task 1 is regression, aiming to predict the student's final grade, `G3` (a number from 0 to 20).
2. Task 2 is classification, aiming to determine whether the student is in a `romantic` relationship or not.
   
This notebook fully covers all the code and experiments included in this repository through Python scripts. The interpreter used to run the code snippets in this notebook was 3.14; it was the same interpreter used for the entire project. After ensuring that you are using Python 3.14, install all the necessary libraries for this project:

In [ ]:
!pip install -U matplotlib numpy pandas scikit-learn torch ucimlrepo

Here I will list all the imports used throughout the notebook (and project) so that I will not need to need to run any functional code snippet twice:

In [ ]:
import logging
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import torch
import torch.nn as nn

from collections import Counter
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, accuracy_score, f1_score, classification_report, confusion_matrix, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from ucimlrepo import fetch_ucirepo

Now let's start by using UC Irvine's Python API to retrieve the "Student Performance" dataset:

In [ ]:
student_performance = fetch_ucirepo(name="Student Performance")
student_performance.metadata.additional_info.summary

### Attributes of the dataset:
1. school: student's school (binary: "GP" - Gabriel Pereira or "MS" - Mousinho da Silveira)
2. sex: student's sex (binary: "F" - female or "M" - male)
3. age: student's age (numeric: from 15 to 22)
4. address: student's home address type (binary: "U" - urban or "R" - rural)
5. famsize: family size (binary: "LE3" - less or equal to 3 or "GT3" - greater than 3)
6. Pstatus: parent's cohabitation status (binary: "T" - living together or "A" - apart)
7. Medu: mother's education (numeric: 0 - none,  1 - primary education (4th grade), 2 - 5th to 9th grade, 3 - secondary education or 4 - higher education)
8. Fedu: father's education (numeric: 0 - none,  1 - primary education (4th grade), 2 - 5th to 9th grade, 3 - secondary education or 4 - higher education)
9. Mjob: mother's job (nominal: "teacher", "health" care related, civil "services" (e.g. administrative or police), "at_home" or "other")
10. Fjob: father's job (nominal: "teacher", "health" care related, civil "services" (e.g. administrative or police), "at_home" or "other")
11. reason: reason to choose this school (nominal: close to "home", school "reputation", "course" preference or "other")
12. guardian: student's guardian (nominal: "mother", "father" or "other")
13. traveltime: home to school travel time (numeric: 1 - <15 min., 2 - 15 to 30 min., 3 - 30 min. to 1 hour, or 4 - >1 hour)
14. studytime: weekly study time (numeric: 1 - <2 hours, 2 - 2 to 5 hours, 3 - 5 to 10 hours, or 4 - >10 hours)
15. failures: number of past class failures (numeric: n if $1 \leq n <3$, else 4)
16. schoolsup: extra educational support (binary: yes or no)
17. famsup: family educational support (binary: yes or no)
18. paid: extra paid classes within the course subject (Math or Portuguese) (binary: yes or no)
19. activities: extra-curricular activities (binary: yes or no)
20. nursery: attended nursery school (binary: yes or no)
21. higher: wants to take higher education (binary: yes or no)
22. internet: Internet access at home (binary: yes or no)
23. ***romantic***: with a romantic relationship (binary: yes or no) [TARGET]
24. famrel: quality of family relationships (numeric: from 1 - very bad to 5 - excellent)
25. freetime: free time after school (numeric: from 1 - very low to 5 - very high)
26. goout: going out with friends (numeric: from 1 - very low to 5 - very high)
27. Dalc: workday alcohol consumption (numeric: from 1 - very low to 5 - very high)
28. Walc: weekend alcohol consumption (numeric: from 1 - very low to 5 - very high)
29. health: current health status (numeric: from 1 - very bad to 5 - very good)
30. absences: number of school absences (numeric: from 0 to 93)

These grades are related with the course subject, Math or Portuguese:

31. G1: first period grade (numeric: from 0 to 20)
31. G2: second period grade (numeric: from 0 to 20)
32. ***G3***: final grade (numeric: from 0 to 20) [TARGET]

## Part 1: Data Preprocessing (`preprocessing.py`)

Just like the majority of machine learning models, neural network performance heavily relies on the quality of the data used to train and evaluate it. Additionally, neural networks only accept numeric data as input, so categorical data needs to be transformed into numerical format as well.

Usually, one would check for missing values, but since the UCI website states that **there are no missing values** in the dataset, we can skip this step.

In this order, we will:
1. Separate target variables
2. Perform a train/test split
3. Encode categorical variables into numerical ones
4. Normalize/standardize numerical variables

First, we separate the target variables from the dataset. Now, grades (`G1`, `G2`, `G3`) are in `student_performance.data.targets`, while the rest are under `student_performance.data.features`.

From the [webpage for the dataset](https://archive.ics.uci.edu/dataset/320/student+performance), we read:
> Important note: the target attribute G3 has a strong correlation with attributes G2 and G1. This occurs because G3 is the final year grade (issued at the 3rd period), while G1 and G2 correspond to the 1st and 2nd period grades. It is more difficult to predict G3 without G2 and G1, but such prediction is much more useful (see paper source for more details).

So we will ignore `G1` and `G2` altogether, separating `romantic`, `G3`, and all the remaining features of `student_performance.data.features`:

In [ ]:
G3 = student_performance.data.targets["G3"]
romantic = student_performance.data.features["romantic"]
X = student_performance.data.features.drop(columns=["romantic"])
y = pd.DataFrame({"G3": G3, "romantic": romantic})
og_columns = [col for col in X.columns]

# is "romantic" disbalanced? yes, it is
y["romantic"].value_counts()

Now, we perform a train/test split. `random_state` for reproducibility (95 for Lightning McQueen); `stratify` to ensure balanced distribution for `romantic`:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=95, stratify=y["romantic"])

G3_train, G3_test = y_train["G3"], y_test["G3"]
romantic_train, romantic_test = y_train["romantic"], y_test["romantic"]

We will now check which of our variables are categorical and which are numerical, and then start by converting the categorical variables into numerical format:

In [ ]:
for col in X_train.columns:
    print(f"{col}: {X_train[col].dtype}")

As we can see, for this particular dataset, the datatype for all of the categorical variables is `"object"`, so we can proceed with the following:

In [ ]:
categorical_columns = []
numerical_columns = []
for col in X_train.columns:
    if X_train[col].dtype == "object":
        categorical_columns.append(col)
    else:
        numerical_columns.append(col)

print(categorical_columns)
print(numerical_columns)

For both categorical and numerical variables, we fit the encoder, normalizer, or standardizer on `X_train`, and then apply the fitted method to `X_test` to prevent data leakage.

As for the strategy of encoding categorical variables:
- For binary categorical variables, binary encoding doesn't create artificial ordinal relationships. Each variable will simply map to 0 or 1.
- For nominal variables (`Mjob`, `Fjob`, `reason`, `guardian`), one-hot encoding creates binary columns for each category, preventing the model from assuming false ordinal relationships (e.g., "teacher" > "health"). This is not an overhead, as none of these variables take more than four possible values.
- Ordinal categorical variables are numerical features for this dataset. We standardize all numerical columns. This is necessary because even though ordinal variables have meaningful ordering, they exist on different scales (some 0-4, others 1-5, and `absences` goes 0-93), which would cause the neural network to give disproportionate weight to larger-scale features during gradient descent. Standardization puts all features on the same scale ($\mu=0, \sigma=1$) so the network can learn their relative importance based on predictive power, not magnitude.

In [ ]:
# Binary categorical variables (encode as 0/1)
binary_columns = ["school", "sex", "address", "famsize", "Pstatus", "schoolsup", "famsup", "paid", "activities", "nursery", "higher", "internet"]
# Nominal categorical variables (one-hot encode)
nominal_columns = ["Mjob", "Fjob", "reason", "guardian"]

# Create the preprocessor
column_transformer = ColumnTransformer(
    transformers=[
        ("binary", OrdinalEncoder(), binary_columns),
        ("nominal", OneHotEncoder(drop="first", sparse_output=False, handle_unknown="ignore"), nominal_columns),
        ("numerical", StandardScaler(), numerical_columns),
    ]
)

# Fit on training data ONLY
column_transformer.fit(X_train)

# Transform both training and test sets
X_train_encoded = column_transformer.transform(X_train)
X_test_encoded = column_transformer.transform(X_test)

Converting back to DataFrame for easier inspection:

In [ ]:
feature_names = (
    binary_columns + 
    column_transformer.named_transformers_["nominal"].get_feature_names_out(nominal_columns).tolist() +
    numerical_columns
)

X_train_encoded = pd.DataFrame(X_train_encoded, columns=feature_names, index=X_train.index)
X_test_encoded = pd.DataFrame(X_test_encoded, columns=feature_names, index=X_test.index)

Let's create a custom PyTorch dataset (`CrushSet.py`):

In [ ]:
class CrushSet(Dataset):
    def __init__(self, X: pd.DataFrame, y: pd.DataFrame):
        """
        Create a dataset for heartbroken nerds. (Not for me I swear, I have the most beautiful and loving girlfriend in the history of the world)

        Args:
            X: (n_samples, n_features) -> preprocessed features
            y: (n_samples, 2) -> targets DataFrame containing both 'G3' and 'romantic' columns
        """
        self.X = torch.FloatTensor(X.values)
        self.y_G3 = torch.FloatTensor(y["G3"].values)
        self.y_romantic = torch.LongTensor((y["romantic"] == "yes").astype(int).values)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        """
        Returns three items: (x_data, y_G3_data, y_romantic_data)

        Returns:
            x_data: feature vector, shape (n_features,)
            y_G3_data: grade target (scalar)
            y_romantic_data: romantic status target (0 or 1)
        """
        return self.X[idx], self.y_G3[idx], self.y_romantic[idx]

Completing train/test/validation split and creating DataLoaders (this snippet of code is implemented in `train.py`, which we'll cover later, but here we can avoid saving the split results and reloading them.):

In [ ]:
# Split train into train and validation sets (80/20 split of training data)
X_train_final, X_val, y_train_final, y_val = train_test_split(X_train_encoded, y_train, test_size=0.2, random_state=95, stratify=y_train["romantic"])

# Create Dataset instances
train_dataset = CrushSet(X_train_final, y_train_final)
test_dataset = CrushSet(X_test_encoded, y_test)
val_dataset = CrushSet(X_val, y_val)

# Create DataLoaders
BS = 32  # yeah, let's go with the default batch size

train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BS, shuffle=False)

## Part 2: Building the Multi-Head Model (`TwoRabbitsHunter.py`)

In this task, we will create a single neural network with a shared feature-extractor("body") and two task-specific output modules("heads").

- Shared body: a feedforward feature extractor that transforms input features into a compact shared representation using repeated blocks of linear projection, nonlinearity, normalization, and regularization. This body is responsible for learning the common student-profile features useful to both tasks.

- First Head: Grade prediction (regression) - a small feedforward module that takes the shared representation and produces one continuous scalar - the predicted grade.

- Second Head: Romantic status (classification) - a small feedforward module that takes the same shared representation and produces two output scores (logits) corresponding to the "yes"/"no" classes.

- Data flow: inputs go into the shared body; its output is forwarded in parallel to both heads; the model returns both the scalar regression output and the two-class logits.

I will name the class `TwoRabbitsHunter`, after a Georgian saying that roughly translates to: 
>A hunter who chases two rabbits will catch neither.

In [ ]:
class TwoRabbitsHunter(nn.Module):
    HEAD_SIZE = 6
    
    def __init__(self, input_size=38, hidden_size=16, shared_output_size=8, dropout_rate=0.3):
        """
        Class to simultaneously predict G3 (regression) and romantic (classification)

        Args:
            input_size: Number of input features (X_train_encoded.shape[1] == 38)
            hidden_size: Size of hidden layers in shared body
            shared_output_size: Size of final shared representation
            dropout_rate: Dropout probability for regularization
        """
        super().__init__()
        # shared body
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(dropout_rate)

        self.fc2 = nn.Linear(hidden_size, shared_output_size)
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm1d(shared_output_size)  # this is where the shared body ends

        # regression head
        self.G3_fc1 = nn.Linear(shared_output_size, self.HEAD_SIZE)
        self.G3_relu = nn.ReLU()
        self.G3_bn = nn.BatchNorm1d(self.HEAD_SIZE)
        self.G3_dropout = nn.Dropout(dropout_rate)
        self.G3_fc2 = nn.Linear(self.HEAD_SIZE, 1)  # G3

        # classification head
        self.romantic_fc1 = nn.Linear(shared_output_size, self.HEAD_SIZE)
        self.romantic_relu = nn.ReLU()
        self.romantic_bn = nn.BatchNorm1d(self.HEAD_SIZE)
        self.romantic_dropout = nn.Dropout(dropout_rate)
        self.romantic_fc2 = nn.Linear(self.HEAD_SIZE, 2)  # romantic logits ("yes"/"no")


    def forward(self, x):
        x = self.dropout1(self.bn1(self.relu1(self.fc1(x))))
        x = self.bn2(self.relu2(self.fc2(x)))

        G3_pred = self.G3_fc2(self.G3_dropout(self.G3_bn(self.G3_relu(self.G3_fc1(x)))))
        romantic_logits = self.romantic_fc2(self.romantic_dropout(self.romantic_bn(self.romantic_relu(self.romantic_fc1(x)))))

        return G3_pred, romantic_logits

As `X_train_encoded.shape = (519, 38)`, a compact two-layer shared body ($38 \to 16 \to $ `HEAD_SIZE`) prevents overfitting while maintaining sufficient representational capacity. The parameter count of $\approx 1,500$ aligns better with the dataset size compared to deeper alternatives. Each head uses a single hidden layer of `HEAD_SIZE` neurons before outputting predictions. A single dropout layer after the first shared layer provides regularization at the point of highest dimensionality where overfitting risk is greatest, while the second layer's batch normalization alone sufficiently regularizes the smaller `HEAD_SIZE`-dimensional representation that feeds into both specialized heads. Dropout rate of $0.2$ is a classic.

## Part 3: The Custom Training Loop (`train.py`)

We will implement a training and validation procedure tailored for a multi-task model with two separate objectives.

Training loop:
- Define two appropriate loss functions: one for the regression task and one for the classification task.
- For each training batch, run a forward pass and obtain two outputs from the model: the regression prediction and the classification logits.
- Compute each task's loss separately using the corresponding predictions and targets, taking care that targets and predictions have the correct shapes and types for each loss.
- Combine the two task losses into a single scalar `total_loss` (for example by summing them, or using weighted sum if we choose to weight tasks differently).
- Backpropagate once on `total_loss` so gradients flow through both heads and the shared body together.
- Update model parameters based on the aggregated gradients.

Validation loop:
- Run the model on validation data in evaluation mode and obtain both outputs per batch.
- Compute each task's validation loss separately, accumulate them across the validation set, and report both validation losses (and optionally their combined value).
- Use these validation losses to monitor training progress and for early stopping or hyperparameter decisions.

But before the loops, let's configure logger:

In [ ]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)

For the sake of training the model for different values of $\alpha$, I have to put everything together in a single `train_model(alpha)` method:

In [ ]:
def train_model(alpha, X_train_final, y_train_final, X_val, y_val, X_test_encoded, y_test):
    logger.info("=" * 50)
    logger.info(f"Training model with alpha = {alpha}")
    logger.info("=" * 50)

    # create Dataset instances using provided data
    train_dataset = CrushSet(X_train_final, y_train_final)
    test_dataset = CrushSet(X_test_encoded, y_test)
    val_dataset = CrushSet(X_val, y_val)

    # create DataLoaders
    BS = 32

    # compute weights for class imbalance
    romantic_labels = [int(y) for _, _, y in train_dataset]
    class_counts = Counter(romantic_labels)
    total = sum(class_counts.values())
    weights = [total / class_counts[i] for i in range(2)]
    sample_weights = [weights[y] for y in romantic_labels]
    sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

    train_loader = DataLoader(train_dataset, batch_size=BS, sampler=sampler)
    val_loader = DataLoader(val_dataset, batch_size=BS, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=BS, shuffle=False)

    # model initialization
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = TwoRabbitsHunter(input_size=X_train_final.shape[1]).to(device)

    regression_loss_fn = nn.MSELoss()
    class_weights = torch.tensor(weights, dtype=torch.float).to(device)
    classification_loss_fn = nn.CrossEntropyLoss(weight=class_weights)

    learning_rate = 0.0005
    weight_decay = 1e-4
    epochs = 500

    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

    logger.info(f"Alpha (G3 weight): {alpha}")
    logger.info(f"Using device: {device}")

    # lists to track metrics for plotting
    train_total_losses = []
    train_G3_losses = []
    train_romantic_losses = []
    val_total_losses = []
    val_G3_losses = []
    val_romantic_losses = []

    # training loop
    for epoch in range(epochs):
        model.train()
        train_G3_loss = 0.0
        train_romantic_loss = 0.0
        train_total_loss = 0.0
        train_batches = 0.0

        for batch_X, batch_G3, batch_romantic in train_loader:
            batch_X = batch_X.to(device)
            batch_G3 = batch_G3.to(device)
            batch_romantic = batch_romantic.to(device)

            optimizer.zero_grad()

            G3_pred, romantic_logits = model(batch_X)

            G3_loss = regression_loss_fn(G3_pred.squeeze(), batch_G3)
            romantic_loss = classification_loss_fn(romantic_logits, batch_romantic)
            total_loss = alpha * G3_loss + (1 - alpha) * romantic_loss

            total_loss.backward()
            optimizer.step()

            train_G3_loss += G3_loss.item()
            train_romantic_loss += romantic_loss.item()
            train_total_loss += total_loss.item()
            train_batches += 1

        avg_train_G3 = train_G3_loss / train_batches
        avg_train_romantic = train_romantic_loss / train_batches
        avg_train_total = train_total_loss / train_batches

        train_total_losses.append(avg_train_total)
        train_G3_losses.append(avg_train_G3)
        train_romantic_losses.append(avg_train_romantic)

        # validation
        model.eval()
        val_G3_loss = 0.0
        val_romantic_loss = 0.0
        val_total_loss = 0.0
        val_batches = 0.0

        with torch.no_grad():
            for batch_X, batch_G3, batch_romantic in val_loader:
                batch_X = batch_X.to(device)
                batch_G3 = batch_G3.to(device)
                batch_romantic = batch_romantic.to(device)

                G3_pred, romantic_logits = model(batch_X)

                G3_loss = regression_loss_fn(G3_pred.squeeze(), batch_G3)
                romantic_loss = classification_loss_fn(romantic_logits, batch_romantic)
                total_loss = alpha * G3_loss + (1 - alpha) * romantic_loss

                val_G3_loss += G3_loss.item()
                val_romantic_loss += romantic_loss.item()
                val_total_loss += total_loss.item()
                val_batches += 1

        avg_val_total = val_total_loss / val_batches
        avg_val_G3 = val_G3_loss / val_batches
        avg_val_romantic = val_romantic_loss / val_batches

        val_total_losses.append(avg_val_total)
        val_G3_losses.append(avg_val_G3)
        val_romantic_losses.append(avg_val_romantic)

        if (epoch + 1) % 10 == 0:
            logger.info(f"Epoch [{epoch + 1}/{epochs}]")
            logger.info(f"  Train -> Total: {avg_train_total:.3f}, G3: {avg_train_G3:.3f}, Romantic: {avg_train_romantic:.3f}")
            logger.info(f"  Val -> Total: {avg_val_total:.3f}, G3: {avg_val_G3:.3f}, Romantic: {avg_val_romantic:.3f}")

    # don't save model - as opposed to `train.py` - return it directly
    logger.info(f"Training completed for alpha={alpha}")

    metrics: dict[str, list[float]] = {
        'train_total': train_total_losses,
        'train_G3': train_G3_losses,
        'train_romantic': train_romantic_losses,
        'val_total': val_total_losses,
        'val_G3': val_G3_losses,
        'val_romantic': val_romantic_losses,
        'class_weights': weights
    }

    return model, metrics

Now, let's use the method above $\forall \alpha \in \{0.3, 0.5, 0.7\}$:

In [ ]:
alpha_values = [0.3, 0.5, 0.7]
models = {}
model_metrics = {}

for i, alpha in enumerate(alpha_values):
    models[alpha], model_metrics[alpha] = train_model(alpha, X_train_final, y_train_final, X_val, y_val, X_test_encoded, y_test)

logger.info("All training completed!")

## Part 4: Evaluation & Analysis

A single "accuracy" score is meaningless. We must evaluate each task's performance on test set. 

- For `G3` prediction:
  - We report the Mean Absolute Error (MAE). This tells us, "On average, how many grade points was our model's prediction off by?". We use MAE because its scale is more intuitive than MSE's (humans tend to comprehend the average difference in grade points better than the squared metric of MSE).
  
- For `romantic` prediction:
  - We report accuracy.
  - Additionally, we report the F1-Score (for the "yes" class). This is crucial, as the classes are imbalanced and it balances precision and recall.

Again, for the sake of evaluating for different values of $\alpha$, I will put everything together in one method:

In [ ]:
def evaluate_model(alpha):
    logger.info("=" * 50)
    logger.info(f"Evaluating model with alpha = {alpha}")
    logger.info("=" * 50)

    model = TwoRabbitsHunter(input_size=X_test_encoded.shape[1]).to(device)
    model.load_state_dict(torch.load(f"tmp/TwoRabbitsHunter_{alpha}.pth", map_location=device))
    model.eval()

    regression_loss_fn = nn.MSELoss()
    with open(f"tmp/metrics_alpha_{alpha}.pkl", 'rb') as f:
        train_data = pickle.load(f)
    class_weights = torch.tensor(train_data.get('class_weights', [1.0, 1.0]), dtype=torch.float).to(device)
    classification_loss_fn = nn.CrossEntropyLoss(weight=class_weights)

    all_G3_preds = []
    all_G3_true = []
    all_romantic_preds = []
    all_romantic_true = []

    test_total_losses = []
    test_G3_losses = []
    test_romantic_losses = []

    with torch.no_grad():
        for batch_X, batch_G3, batch_romantic in test_loader:
            batch_X = batch_X.to(device)
            batch_G3 = batch_G3.to(device)
            batch_romantic = batch_romantic.to(device)

            G3_pred, romantic_logits = model(batch_X)

            G3_loss = regression_loss_fn(G3_pred.squeeze(), batch_G3)
            romantic_loss = classification_loss_fn(romantic_logits, batch_romantic)
            total_loss = alpha * G3_loss + (1 - alpha) * romantic_loss

            test_total_losses.append(total_loss.item())
            test_G3_losses.append(G3_loss.item())
            test_romantic_losses.append(romantic_loss.item())

            all_G3_preds.extend(G3_pred.squeeze().cpu().numpy())
            all_G3_true.extend(batch_G3.cpu().numpy())

            romantic_pred_classes = torch.argmax(romantic_logits, dim=1)
            all_romantic_preds.extend(romantic_pred_classes.cpu().numpy())
            all_romantic_true.extend(batch_romantic.cpu().numpy())

    avg_test_total = sum(test_total_losses) / len(test_total_losses)
    avg_test_G3 = sum(test_G3_losses) / len(test_G3_losses)
    avg_test_romantic = sum(test_romantic_losses) / len(test_romantic_losses)

    all_G3_preds = np.array(all_G3_preds)
    all_G3_true = np.array(all_G3_true)
    all_romantic_preds = np.array(all_romantic_preds)
    all_romantic_true = np.array(all_romantic_true)

    mae_G3 = mean_absolute_error(all_G3_true, all_G3_preds)
    accuracy_romantic = accuracy_score(all_romantic_true, all_romantic_preds)
    f1_romantic_yes = f1_score(all_romantic_true, all_romantic_preds, pos_label=1)
    f1_romantic_no = f1_score(all_romantic_true, all_romantic_preds, pos_label=0)
    unique, counts = np.unique(all_romantic_true, return_counts=True)

    logger.info("Test Set Class Distribution:")
    for cls, cnt in zip(unique, counts):
        class_name = "yes" if cls == 1 else "no"
        logger.info(f"  {class_name}: {cnt} samples ({cnt / len(all_romantic_true) * 100:.1f}%)")

    logger.info("=" * 50)
    logger.info(f"SUMMARY FOR ALPHA = {alpha}")
    logger.info("=" * 50)
    logger.info(f"G3 Prediction MAE: {mae_G3:.3f} points")
    logger.info(f"Interpretation: On average, predictions are off by {mae_G3:.3f} points (out of 20)")
    logger.info(f"Romantic Accuracy: {accuracy_romantic:.3f} ({accuracy_romantic * 100:.1f}%)")
    logger.info(f"Romantic F1-Score for 'yes' class: {f1_romantic_yes:.3f}")
    logger.info(f"Romantic F1-Score for 'no' class: {f1_romantic_no:.3f}")
    logger.info("=" * 50)

    metrics: dict[str, float | list[float] | dict] = {
        'test_total_loss': avg_test_total,
        'test_G3_loss': avg_test_G3,
        'test_romantic_loss': avg_test_romantic,
        'test_total_losses': test_total_losses,
        'test_G3_losses': test_G3_losses,
        'test_romantic_losses': test_romantic_losses,
        'mae_G3': mae_G3,
        'accuracy_romantic': accuracy_romantic,
        'f1_romantic_yes': f1_romantic_yes,
        'f1_romantic_no': f1_romantic_no,
        'class_distribution': dict(zip(unique, counts))
    }

    return metrics

We also need plotting method. I used DeepSeek to write this method, as I am not great at plotting:

In [ ]:
def plot_train_vs_test_comparison(alpha, test_metrics, save_plot=True):
    try:
        with open(f"tmp/metrics_alpha_{alpha}.pkl", 'rb') as f:
            train_data = pickle.load(f)
    except FileNotFoundError:
        logger.error(f"Metrics file for alpha={alpha} not found. Run training first.")
        return

    plt.style.use('dark_background')
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    epochs = range(1, len(train_data['train_total']) + 1)
    
    axes[0].plot(epochs, train_data['train_total'], 'purple', linewidth=2, label='Training')
    axes[0].plot(epochs, train_data['val_total'], 'lime', linewidth=2, label='Validation')
    axes[0].axhline(y=test_metrics['test_total_loss'], color='cyan', linestyle='--', linewidth=2, label='Test')
    axes[0].set_title(f'Total Loss (Alpha={alpha})')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(epochs, train_data['train_G3'], 'purple', linewidth=2, label='Training')
    axes[1].plot(epochs, train_data['val_G3'], 'lime', linewidth=2, label='Validation')
    axes[1].axhline(y=test_metrics['test_G3_loss'], color='cyan', linestyle='--', linewidth=2, label='Test')
    axes[1].set_title(f'G3 Loss (Alpha={alpha})')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    axes[2].plot(epochs, train_data['train_romantic'], 'purple', linewidth=2, label='Training')
    axes[2].plot(epochs, train_data['val_romantic'], 'lime', linewidth=2, label='Validation')
    axes[2].axhline(y=test_metrics['test_romantic_loss'], color='cyan', linestyle='--', linewidth=2, label='Test')
    axes[2].set_title(f'Romantic Loss (Alpha={alpha})')
    axes[2].set_xlabel('Epoch')
    axes[2].set_ylabel('Loss')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)

    plt.tight_layout()

    if save_plot:
        os.makedirs('tmp', exist_ok=True)
        plot_filename = f"tmp/train_val_test_losses_alpha_{alpha}.png"
        plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
        plt.close()
        logger.info(f"Training vs Validation vs Test loss curves saved to '{plot_filename}'")
    else:
        plt.show()

    final_train = {
        'total_loss': train_data['train_total'][-1],
        'G3_loss': train_data['train_G3'][-1],
        'romantic_loss': train_data['train_romantic'][-1]
    }
    final_val = {
        'total_loss': train_data['val_total'][-1],
        'G3_loss': train_data['val_G3'][-1],
        'romantic_loss': train_data['val_romantic'][-1]
    }

    logger.info("=" * 60)
    logger.info(f"FINAL LOSS COMPARISON FOR ALPHA = {alpha}")
    logger.info("=" * 60)
    logger.info("Total Loss:")
    logger.info(f"  Training: {final_train['total_loss']:.3f}")
    logger.info(f"  Validation: {final_val['total_loss']:.3f}")
    logger.info(f"  Test: {test_metrics['test_total_loss']:.3f}")
    logger.info("G3 Loss:")
    logger.info(f"  Training: {final_train['G3_loss']:.3f}")
    logger.info(f"  Validation: {final_val['G3_loss']:.3f}")
    logger.info(f"  Test: {test_metrics['test_G3_loss']:.3f}")
    logger.info("Romantic Loss:")
    logger.info(f"  Training: {final_train['romantic_loss']:.3f}")
    logger.info(f"  Validation: {final_val['romantic_loss']:.3f}")
    logger.info(f"  Test: {test_metrics['test_romantic_loss']:.3f}")
    logger.info("=" * 60)

    return fig

In [ ]:
alpha_values = [0.3, 0.5, 0.7]
evaluated_metrics = {}  # not really useful rn

for alpha in alpha_values:
    test_metrics = evaluate_model(alpha)
    evaluated_metrics[alpha] = test_metrics
    plot_train_vs_test_comparison(alpha, test_metrics, save_plot=True)

logger.info("All evaluation and comparison plotting completed!"

## Summary

Based on the evaluation of three multi-task models with different alpha values - the parameter that balances the weight between the grade regression (`G3`) and romance classification tasks - a clear performance hierarchy is established, with $\alpha=0.5$ emerging as the optimal configuration.

The model with $\alpha=0.3$ places less emphasis on the grade prediction task. This results in the highest test MAE of $2.066$ for grades and a higher total test loss compared to $\alpha=0.5$, suggesting its weighting is suboptimal for the primary objective.

The model with $\alpha=0.5$ demonstrates the best balance. It achieves the lowest Grade Prediction MAE of $1.931$ on the test set, indicating the most accurate forecasts. Its total test loss of $3.735$ is the lowest among all models, and it exhibits a healthy generalization pattern where validation and test losses are lower than training loss. This indicates that the auxiliary romance task, while not producing meaningful classifications itself, provides a beneficial regularization effect at this weight.

Conversely, the model with $\alpha=0.7$ heavily prioritizes the grade task. However, this leads to overfitting, evidenced by its significantly higher total test loss of $5.099$. While its grade MAE of $1.943$ is better than $\alpha=0.3$, it is still worse than $\alpha=0.5$. The romantic loss also shows a larger gap between training and test performance, further confirming overfitting. This implies that completely overshadowing the auxiliary task removes its regularizing benefit, degrading overall model robustness.

In conclusion, while the romance classifier failed universally, its influence as a regularizer is crucial. The balanced weighting at $\alpha=0.5$ provides the ideal compromise, yielding the most accurate and generalizable model for the primary task of grade prediction, making it the definitive best choice.